In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import seaborn as sns
import gc
from tqdm import tqdm

%matplotlib inline

PATH = ''

In [2]:
%%time
train = pd.read_csv(PATH + 'train.csv')
test = pd.read_csv(PATH + 'Track 1/test.csv')
submission = pd.read_csv(PATH + 'Track 1/submission.csv')

Wall time: 2.55 s


In [12]:
def smape(satellite_predicted_values, satellite_true_values): 
    # the division, addition and subtraction are pointwise 
    return np.mean(np.abs(satellite_predicted_values - satellite_true_values) / (np.abs(satellite_predicted_values) + np.abs(satellite_true_values)))

def getLBScore(real, predicted):
    smp = []
    for sat_id in real['sat_id'].unique():
        rv = real[real['sat_id'] == sat_id][['x', 'y', 'z', 'Vx', 'Vy', 'Vz']].values
        sym = predicted[predicted['sat_id'] == sat_id][['x', 'y', 'z', 'Vx', 'Vy', 'Vz']].values
        smp.append(smape(sym.ravel(), rv.ravel()))
    meanSmape = np.mean(smp)
    lbScore = 100 * (1 - meanSmape)
    print('Mean SMAPE ', meanSmape, '; LB score ', lbScore)

In [4]:
training_columns = ['epoch', 'sat_id', 'x_sim', 'y_sim', 'z_sim', 'Vx_sim', 'Vy_sim', 'Vz_sim']
prediction_columns = ['x','y','z','Vx', 'Vy', 'Vz']
X = train.loc[:, training_columns]
Y = train.loc[:, prediction_columns]
Xtest = test.loc[:, training_columns]

In [6]:
from sklearn.model_selection import train_test_split

Xtr, Xval, ytr, yval = train_test_split(X, Y, train_size = 0.9)
submissionValid = train.iloc[Xval.index, :].copy()[['id', 'sat_id', 'x', 'y', 'z', 'Vx', 'Vy', 'Vz']]
submissionValid = submissionValid.reset_index(drop=True)

C:\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [9]:
from sklearn.model_selection import train_test_split

Xtr, Xval, ytr, yval = train_test_split(X, Y, train_size = 0.9)
submissionValid = train.iloc[Xval.index, :].copy()[['id', 'sat_id', 'x', 'y', 'z', 'Vx', 'Vy', 'Vz']]
submissionValid = submissionValid.reset_index(drop=True)
idx = 0
for idx in tqdm(range(Xval.shape[0])):
    row = Xval.iloc[idx, :]
    trData = Xtr[Xtr['sat_id'] == row['sat_id']]
    trValues = ytr[Xtr['sat_id'] == row['sat_id']]
    x,y,z = row.iloc[[2,3,4]].values
    minIndex = np.argmin(np.sqrt(np.sum((trData.iloc[:, [2,3,4]].values - [x,y,z])**2, axis = 1)))
    submissionValid.iloc[idx, 2:] = trData.iloc[minIndex, 2:8].values

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 64992/64992 [07:46<00:00, 139.25it/s]


In [13]:
getLBScore(train.iloc[Xval.index, :].copy()[['id', 'sat_id', 'x', 'y', 'z', 'Vx', 'Vy', 'Vz']], submissionValid)

Mean SMAPE  0.30104707832278793 ; LB score  69.89529216772121


In [32]:
idx = 0
for sat_id in tqdm(sorted(test['sat_id'].unique())):
    trData = X[satellites == sat_id]
    trValues = Y[satellites == sat_id]
    
    testData = test[test['sat_id'] == sat_id]
    for index in range(testData.shape[0]):
        x,y,z = testData.iloc[index, [3,4,5]].values
        minIndex = np.argmin(np.sqrt(np.sum((trData.iloc[:, [2,3,4]].values - [x,y,z])**2, axis = 1)))
        submission.iloc[idx, 1:] = trData.iloc[minIndex, 2:8].values
        idx+=1

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 300/300 [51:51<00:00,  9.17s/it]


In [33]:
# idx = 0
# satellites = X['sat_id']
# for idx in tqdm(range(test.shape[0])):
#     row = test.iloc[idx, :]
#     trData = X[satellites == row['sat_id']]
#     trValues = Y[satellites == row['sat_id']]
#     x,y,z = row.iloc[[3,4,5]].values
#     minIndex = np.argmin(np.sqrt(np.sum((trData.iloc[:, [2,3,4]].values - [x,y,z])**2, axis = 1)))
#     submission.iloc[idx, 1:] = trData.iloc[minIndex, 2:8].values

In [34]:
submission.to_csv('submission.csv', index = None)